This notebook takes from the tutorial at https://docs.lsdb.io/en/stable/tutorials/pre_executed/des-gaia.html. I intend to manipulate it as necessary to crossmatch the data used in previous versions of des_gaia_xmatch.ipynb, and perform the data pipeline.

In [1]:
from pathlib import Path

import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM

In [2]:
# Change to the directories where the data will be stored
DES_DIR = Path("../../lsdb_tutorials/data/DES_DR2")
GAIA_DIR = Path("../../lsdb_tutorials/data/Gaia_DR3")

GAIA_SCHEMA_FILE = GAIA_DIR / "schema.parquet"

DES_HATS_NAME = "des_dr2"
GAIA_HATS_NAME = "gaia_dr3"
GAIA_MARGIN_CACHE_NAME = "gaia_dr3_1arcsec"
XMATCH_NAME = "des_dr2_x_gaia_dr3"

HATS_DIR = Path("../../lsdb_tutorials/catalogs")
DES_HATS_DIR = HATS_DIR / DES_HATS_NAME
GAIA_HATS_DIR = HATS_DIR / GAIA_HATS_NAME
GAIA_MARGIN_CACHE_DIR = HATS_DIR / GAIA_MARGIN_CACHE_NAME

XMATCH_DIR = HATS_DIR / XMATCH_NAME

In [3]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
k = 1
collinear_error_cutoff = 0.2
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
cone_search_rad = 50
max_neighbor_dist = 18
milliarc_degree_conversion = 1/(1000*3600)

In [4]:
des_catalog = lsdb.read_hats(DES_HATS_DIR)
gaia_catalog = lsdb.read_hats(GAIA_HATS_DIR)
xmatch_catalog = lsdb.read_hats(XMATCH_DIR)

In [5]:
print(type(xmatch_catalog))
print(type(gaia_catalog))
print(type(des_catalog))

<class 'lsdb.catalog.catalog.Catalog'>
<class 'lsdb.catalog.catalog.Catalog'>
<class 'lsdb.catalog.catalog.Catalog'>


In [6]:
pm_filter_xmatch = xmatch_catalog.query(f'{pm_speed_max**2} >(pmra_gaia**2 + pmdec_gaia**2) > {pm_speed_min**2}')
with Client():
    df = pm_filter_xmatch.compute()

df

,COADD_OBJECT_ID_des,TILENAME_des,HPIX_32_des,HPIX_64_des,HPIX_1024_des,HPIX_4096_des,HPIX_16384_des,RA_des,DEC_des,ALPHAWIN_J2000_des,DELTAWIN_J2000_des,GALACTIC_L_des,GALACTIC_B_des,XWIN_IMAGE_des,YWIN_IMAGE_des,A_IMAGE_des,ERRA_IMAGE_des,B_IMAGE_des,ERRB_IMAGE_des,THETA_J2000_des,ERRTHETA_IMAGE_des,KRON_RADIUS_des,EBV_SFD98_des,MAG_AUTO_G_DERED_des,MAG_AUTO_R_DERED_des,MAG_AUTO_I_DERED_des,MAG_AUTO_Z_DERED_des,MAG_AUTO_Y_DERED_des,WAVG_MAG_PSF_G_DERED_des,WAVG_MAG_PSF_R_DERED_des,WAVG_MAG_PSF_I_DERED_des,WAVG_MAG_PSF_Z_DERED_des,WAVG_MAG_PSF_Y_DERED_des,EXTENDED_CLASS_COADD_des,EXTENDED_CLASS_WAVG_des,FLAGS_G_des,IMAFLAGS_ISO_G_des,NEPOCHS_G_des,FLAGS_R_des,IMAFLAGS_ISO_R_des,NEPOCHS_R_des,FLAGS_I_des,IMAFLAGS_ISO_I_des,NEPOCHS_I_des,FLAGS_Z_des,IMAFLAGS_ISO_Z_des,NEPOCHS_Z_des,FLAGS_Y_des,IMAFLAGS_ISO_Y_des,NEPOCHS_Y_des,XWIN_IMAGE_G_des,XWIN_IMAGE_R_des,XWIN_IMAGE_I_des,XWIN_IMAGE_Z_des,XWIN_IMAGE_Y_des,YWIN_IMAGE_G_des,YWIN_IMAGE_R_des,YWIN_IMAGE_I_des,YWIN_IMAGE_Z_des,YWIN_IMAGE_Y_des,X2WIN_IMAGE_G_des,X2WIN_IMAGE_R_des,X2WIN_IMAGE_I_des,X2WIN_IMAGE_Z_des,X2WIN_IMAGE_Y_des,Y2WIN_IMAGE_G_des,Y2WIN_IMAGE_R_des,Y2WIN_IMAGE_I_des,Y2WIN_IMAGE_Z_des,Y2WIN_IMAGE_Y_des,XYWIN_IMAGE_G_des,XYWIN_IMAGE_R_des,XYWIN_IMAGE_I_des,XYWIN_IMAGE_Z_des,XYWIN_IMAGE_Y_des,ERRX2WIN_IMAGE_G_des,ERRX2WIN_IMAGE_R_des,ERRX2WIN_IMAGE_I_des,ERRX2WIN_IMAGE_Z_des,ERRX2WIN_IMAGE_Y_des,ERRY2WIN_IMAGE_G_des,ERRY2WIN_IMAGE_R_des,ERRY2WIN_IMAGE_I_des,ERRY2WIN_IMAGE_Z_des,ERRY2WIN_IMAGE_Y_des,ERRXYWIN_IMAGE_G_des,ERRXYWIN_IMAGE_R_des,ERRXYWIN_IMAGE_I_des,ERRXYWIN_IMAGE_Z_des,ERRXYWIN_IMAGE_Y_des,AWIN_IMAGE_G_des,AWIN_IMAGE_R_des,AWIN_IMAGE_I_des,AWIN_IMAGE_Z_des,AWIN_IMAGE_Y_des,BWIN_IMAGE_G_des,BWIN_IMAGE_R_des,BWIN_IMAGE_I_des,BWIN_IMAGE_Z_des,BWIN_IMAGE_Y_des,THETAWIN_IMAGE_G_des,THETAWIN_IMAGE_R_des,THETAWIN_IMAGE_I_des,THETAWIN_IMAGE_Z_des,THETAWIN_IMAGE_Y_des,ERRAWIN_IMAGE_G_des,ERRAWIN_IMAGE_R_des,ERRAWIN_IMAGE_I_des,ERRAWIN_IMAGE_Z_des,ERRAWIN_IMAGE_Y_des,ERRBWIN_IMAGE_G_des,ERRBWIN_IMAGE_R_des,ERRBWIN_IMAGE_I_des,ERRBWIN_IMAGE_Z_des,ERRBWIN_IMAGE_Y_des,ERRTHETAWIN_IMAGE_G_des,ERRTHETAWIN_IMAGE_R_des,ERRTHETAWIN_IMAGE_I_des,ERRTHETAWIN_IMAGE_Z_des,ERRTHETAWIN_IMAGE_Y_des,FLUX_RADIUS_G_des,FLUX_RADIUS_R_des,FLUX_RADIUS_I_des,FLUX_RADIUS_Z_des,FLUX_RADIUS_Y_des,FWHM_IMAGE_G_des,FWHM_IMAGE_R_des,FWHM_IMAGE_I_des,FWHM_IMAGE_Z_des,FWHM_IMAGE_Y_des,ISOAREA_IMAGE_G_des,ISOAREA_IMAGE_R_des,ISOAREA_IMAGE_I_des,ISOAREA_IMAGE_Z_des,ISOAREA_IMAGE_Y_des,BACKGROUND_G_des,BACKGROUND_R_des,BACKGROUND_I_des,BACKGROUND_Z_des,BACKGROUND_Y_des,NITER_MODEL_G_des,NITER_MODEL_R_des,NITER_MODEL_I_des,NITER_MODEL_Z_des,NITER_MODEL_Y_des,KRON_RADIUS_G_des,KRON_RADIUS_R_des,KRON_RADIUS_I_des,KRON_RADIUS_Z_des,KRON_RADIUS_Y_des,MAG_AUTO_G_des,MAG_AUTO_R_des,MAG_AUTO_I_des,MAG_AUTO_Z_des,MAG_AUTO_Y_des,MAGERR_AUTO_G_des,MAGERR_AUTO_R_des,MAGERR_AUTO_I_des,MAGERR_AUTO_Z_des,MAGERR_AUTO_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,FLUX_AUTO_G_des,FLUX_AUTO_R_des,FLUX_AUTO_I_des,FLUX_AUTO_Z_des,FLUX_AUTO_Y_des,FLUXERR_AUTO_G_des,FLUXERR_AUTO_R_des,FLUXERR_AUTO_I_des,FLUXERR_AUTO_Z_des,FLUXERR_AUTO_Y_des,WAVG_FLUX_PSF_G_des,WAVG_FLUX_PSF_R_des,WAVG_FLUX_PSF_I_des,WAVG_FLUX_PSF_Z_des,WAVG_FLUX_PSF_Y_des,WAVG_FLUXERR_PSF_G_des,WAVG_FLUXERR_PSF_R_des,WAVG_FLUXERR_PSF_I_des,WAVG_FLUXERR_PSF_Z_des,WAVG_FLUXERR_PSF_Y_des,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_SPREAD_MODEL_G_des,WAVG_SPREAD_MODEL_R_des,WAVG_SPREAD_MODEL_I_des,WAVG_SPREAD_MODEL_Z_des,WAVG_SPREAD_MODEL_Y_des,SPREADERR_MODEL_G_des,SPREADERR_MODEL_R_des,SPREADERR_MODEL_I_des,SPREADERR_MODEL_Z_des,SPREADERR_MODEL_Y_des,WAVG_SPREADERR_MODEL_G_des,WAVG_SPREADERR_MODEL_R_des,WAVG_SPREADERR_MODEL_I_des,WAVG_SPREADERR_MODEL_Z

In [7]:
print(type(df))

<class 'nested_pandas.nestedframe.core.NestedFrame'>


In [ ]:
df_no_dupes = df[~df['source_id_gaia'].duplicated(keep='first')]

df_no_dupes

# For some reason this step results in the df having no columns, leading to errors further down the line.
# Since we are currently working with a empty df, I will skip performing this step for now.

""
_healpix_29


In [10]:
#gaia_ids = df_no_dupes['source_id_gaia']

gaia_ids = df['source_id_gaia']

#dropping because otherwise produces error when performing .apply below
# df_no_dupes = df_no_dupes.drop('source_id_gaia', axis=1)
df = df.drop('source_id_gaia', axis=1)

df

,COADD_OBJECT_ID_des,TILENAME_des,HPIX_32_des,HPIX_64_des,HPIX_1024_des,HPIX_4096_des,HPIX_16384_des,RA_des,DEC_des,ALPHAWIN_J2000_des,DELTAWIN_J2000_des,GALACTIC_L_des,GALACTIC_B_des,XWIN_IMAGE_des,YWIN_IMAGE_des,A_IMAGE_des,ERRA_IMAGE_des,B_IMAGE_des,ERRB_IMAGE_des,THETA_J2000_des,ERRTHETA_IMAGE_des,KRON_RADIUS_des,EBV_SFD98_des,MAG_AUTO_G_DERED_des,MAG_AUTO_R_DERED_des,MAG_AUTO_I_DERED_des,MAG_AUTO_Z_DERED_des,MAG_AUTO_Y_DERED_des,WAVG_MAG_PSF_G_DERED_des,WAVG_MAG_PSF_R_DERED_des,WAVG_MAG_PSF_I_DERED_des,WAVG_MAG_PSF_Z_DERED_des,WAVG_MAG_PSF_Y_DERED_des,EXTENDED_CLASS_COADD_des,EXTENDED_CLASS_WAVG_des,FLAGS_G_des,IMAFLAGS_ISO_G_des,NEPOCHS_G_des,FLAGS_R_des,IMAFLAGS_ISO_R_des,NEPOCHS_R_des,FLAGS_I_des,IMAFLAGS_ISO_I_des,NEPOCHS_I_des,FLAGS_Z_des,IMAFLAGS_ISO_Z_des,NEPOCHS_Z_des,FLAGS_Y_des,IMAFLAGS_ISO_Y_des,NEPOCHS_Y_des,XWIN_IMAGE_G_des,XWIN_IMAGE_R_des,XWIN_IMAGE_I_des,XWIN_IMAGE_Z_des,XWIN_IMAGE_Y_des,YWIN_IMAGE_G_des,YWIN_IMAGE_R_des,YWIN_IMAGE_I_des,YWIN_IMAGE_Z_des,YWIN_IMAGE_Y_des,X2WIN_IMAGE_G_des,X2WIN_IMAGE_R_des,X2WIN_IMAGE_I_des,X2WIN_IMAGE_Z_des,X2WIN_IMAGE_Y_des,Y2WIN_IMAGE_G_des,Y2WIN_IMAGE_R_des,Y2WIN_IMAGE_I_des,Y2WIN_IMAGE_Z_des,Y2WIN_IMAGE_Y_des,XYWIN_IMAGE_G_des,XYWIN_IMAGE_R_des,XYWIN_IMAGE_I_des,XYWIN_IMAGE_Z_des,XYWIN_IMAGE_Y_des,ERRX2WIN_IMAGE_G_des,ERRX2WIN_IMAGE_R_des,ERRX2WIN_IMAGE_I_des,ERRX2WIN_IMAGE_Z_des,ERRX2WIN_IMAGE_Y_des,ERRY2WIN_IMAGE_G_des,ERRY2WIN_IMAGE_R_des,ERRY2WIN_IMAGE_I_des,ERRY2WIN_IMAGE_Z_des,ERRY2WIN_IMAGE_Y_des,ERRXYWIN_IMAGE_G_des,ERRXYWIN_IMAGE_R_des,ERRXYWIN_IMAGE_I_des,ERRXYWIN_IMAGE_Z_des,ERRXYWIN_IMAGE_Y_des,AWIN_IMAGE_G_des,AWIN_IMAGE_R_des,AWIN_IMAGE_I_des,AWIN_IMAGE_Z_des,AWIN_IMAGE_Y_des,BWIN_IMAGE_G_des,BWIN_IMAGE_R_des,BWIN_IMAGE_I_des,BWIN_IMAGE_Z_des,BWIN_IMAGE_Y_des,THETAWIN_IMAGE_G_des,THETAWIN_IMAGE_R_des,THETAWIN_IMAGE_I_des,THETAWIN_IMAGE_Z_des,THETAWIN_IMAGE_Y_des,ERRAWIN_IMAGE_G_des,ERRAWIN_IMAGE_R_des,ERRAWIN_IMAGE_I_des,ERRAWIN_IMAGE_Z_des,ERRAWIN_IMAGE_Y_des,ERRBWIN_IMAGE_G_des,ERRBWIN_IMAGE_R_des,ERRBWIN_IMAGE_I_des,ERRBWIN_IMAGE_Z_des,ERRBWIN_IMAGE_Y_des,ERRTHETAWIN_IMAGE_G_des,ERRTHETAWIN_IMAGE_R_des,ERRTHETAWIN_IMAGE_I_des,ERRTHETAWIN_IMAGE_Z_des,ERRTHETAWIN_IMAGE_Y_des,FLUX_RADIUS_G_des,FLUX_RADIUS_R_des,FLUX_RADIUS_I_des,FLUX_RADIUS_Z_des,FLUX_RADIUS_Y_des,FWHM_IMAGE_G_des,FWHM_IMAGE_R_des,FWHM_IMAGE_I_des,FWHM_IMAGE_Z_des,FWHM_IMAGE_Y_des,ISOAREA_IMAGE_G_des,ISOAREA_IMAGE_R_des,ISOAREA_IMAGE_I_des,ISOAREA_IMAGE_Z_des,ISOAREA_IMAGE_Y_des,BACKGROUND_G_des,BACKGROUND_R_des,BACKGROUND_I_des,BACKGROUND_Z_des,BACKGROUND_Y_des,NITER_MODEL_G_des,NITER_MODEL_R_des,NITER_MODEL_I_des,NITER_MODEL_Z_des,NITER_MODEL_Y_des,KRON_RADIUS_G_des,KRON_RADIUS_R_des,KRON_RADIUS_I_des,KRON_RADIUS_Z_des,KRON_RADIUS_Y_des,MAG_AUTO_G_des,MAG_AUTO_R_des,MAG_AUTO_I_des,MAG_AUTO_Z_des,MAG_AUTO_Y_des,MAGERR_AUTO_G_des,MAGERR_AUTO_R_des,MAGERR_AUTO_I_des,MAGERR_AUTO_Z_des,MAGERR_AUTO_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,FLUX_AUTO_G_des,FLUX_AUTO_R_des,FLUX_AUTO_I_des,FLUX_AUTO_Z_des,FLUX_AUTO_Y_des,FLUXERR_AUTO_G_des,FLUXERR_AUTO_R_des,FLUXERR_AUTO_I_des,FLUXERR_AUTO_Z_des,FLUXERR_AUTO_Y_des,WAVG_FLUX_PSF_G_des,WAVG_FLUX_PSF_R_des,WAVG_FLUX_PSF_I_des,WAVG_FLUX_PSF_Z_des,WAVG_FLUX_PSF_Y_des,WAVG_FLUXERR_PSF_G_des,WAVG_FLUXERR_PSF_R_des,WAVG_FLUXERR_PSF_I_des,WAVG_FLUXERR_PSF_Z_des,WAVG_FLUXERR_PSF_Y_des,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_SPREAD_MODEL_G_des,WAVG_SPREAD_MODEL_R_des,WAVG_SPREAD_MODEL_I_des,WAVG_SPREAD_MODEL_Z_des,WAVG_SPREAD_MODEL_Y_des,SPREADERR_MODEL_G_des,SPREADERR_MODEL_R_des,SPREADERR_MODEL_I_des,SPREADERR_MODEL_Z_des,SPREADERR_MODEL_Y_des,WAVG_SPREADERR_MODEL_G_des,WAVG_SPREADERR_MODEL_R_des,WAVG_SPREADERR_MODEL_I_des,WAVG_SPREADERR_MODEL_Z

In [ ]:
def distance_to_line(line_vector, PQ):
    return np.abs(np.cross(PQ, line_vector)) / np.linalg.norm(line_vector)

def within_collinear_error(line_vector, PQ, error):
    return (distance_to_line(line_vector, PQ) <= error)


# Consider making the RA and DEC columns arguments into this func

def kth_star_min_distance(group, k, cols_to_keep):
    origin_ra, origin_dec = group['RA_1'], group['DEC_1']
    ra2, dec2 = group[["RA_2", "DEC_2"]].to_numpy().T
    
    x_vals = (ra2 - origin_ra) * np.cos(np.radians(origin_dec)) * 3600
    y_vals = (dec2 - origin_dec) * 3600
    delta_coords = np.vstack((x_vals, y_vals)).T

    kth_distances = [None] * len(delta_coords)

    for i in range(len(delta_coords)):
        proj_vector = delta_coords[i]
        distances = []

        if np.all(proj_vector == 0): continue #ensures vector points to a point that is not the origin

        for j in range(len(delta_coords)):
            if np.all(delta_coords[j] == 0) or (j == i): continue
            
            distances.append(distance_to_line(proj_vector, delta_coords[j]))

        kth_distances[i] = sorted(distances)[k]
    
    group['kth_min_proj_error'] = kth_distances

    return group[cols_to_keep + ['kth_min_proj_error']]



def algo_found_pm(row, des_cols, cone_search_rad, query_string, max_neighbor_dist, k, collinear_error_cutoff):
    filtered_catalog = (
        lsdb.read_hats(DES_HATS_DIR, columns=des_cols)
        .cone_search(ra=row['ra_gaia'], dec=row['dec_gaia'], radius_arcsec=cone_search_rad)
        .query(query_string)
    )

    xmatch = filtered_catalog.crossmatch(filtered_catalog, n_neighbors=100, radius_arcsec=max_neighbor_dist, suffixes=['_1', '_2'])
    xmatch_ddf = xmatch.ddf

    neighbors = xmatch_ddf.groupby('_hipscat_index')['_dist_arcsec'].count()
    neighbors -= 1 #Double counting adjustment
    neighbors.name = 'neighbors'
    xmatch_ddf = xmatch_ddf.join(neighbors, on='_hipscat_index')
    xmatch_large_groups = xmatch_ddf.query('neighbors >= 3')
    columns_to_keep = (
        [col for col in xmatch_large_groups.columns if col.endswith('_2')]
    )

    meta = make_meta(xmatch_large_groups[columns_to_keep])
    meta['kth_min_proj_error'] = 0.0
    star_groups = xmatch_large_groups.groupby('_hipscat_index').apply(kth_star_min_distance, meta=meta, k=k-1, cols_to_keep = columns_to_keep)  
    df = star_groups.compute().reset_index(drop=True, level = 0)

    
    return bool(not (df.query(f'kth_min_proj_error < {collinear_error_cutoff}').empty))

In [ ]:
#TODO: Update this to fit LSDB updates

df_no_dupes['algo_found'] = df_no_dupes.apply(func=algo_found_pm, axis=1, des_cols=des_cols, cone_search_rad=cone_search_rad, query_string=query_string, max_neighbor_dist=max_neighbor_dist, k=k, collinear_error_cutoff=collinear_error_cutoff)
df_no_dupes['source_id_gaia'] = gaia_ids
df_no_dupes